# MSc Thesis 
# Jacotte Monroe

Code that retrieves automatically the MODIS and Landsat scenes of interest and loads them. 

## Importing and initializing 

In [1]:
import ee
ee.Authenticate()

True

In [2]:
import ee
import geemap
import os
import math
import pandas as pd

ee.Initialize()

## To Change

In [3]:
# define correct week 
week = '2300'

# define type of operation --> '' or 'cloudmasked'
operation_m = 'cloudmasked/'
operation_l = 'cloudmasked/'

# define landsat buffer (days) to make sure there are enough Landsat images 
l8_buffer = ee.Number(30)

## Set-up

In [4]:
# load extent look up table 
extents_lut = pd.read_csv('data/step_extents/random_paths/LA26_step_ex_w' + week + '.csv')

# get largest extent from last row of look up table 
large_extent = extents_lut.iloc[-1]

In [5]:
# retrieve date range from table 
# note: previous week needed to calculate the rate of NDVI change
start_date = ee.Date(large_extent.loc['start_date_prev_week'], 'Africa/Maputo') #ee.Date('2014-01-14') # '2014-02-15'
end_date = ee.Date(large_extent.loc['end_date'], 'Africa/Maputo') #ee.Date('2014-02-18') # '2012-02-17'

In [6]:
# create large extent (region) geometry 
large_extent_coords = [[[large_extent.loc['xmin'], large_extent.loc['ymin']],
                        [large_extent.loc['xmin'], large_extent.loc['ymax']],
                        [large_extent.loc['xmax'], large_extent.loc['ymax']],
                        [large_extent.loc['xmax'], large_extent.loc['ymin']]]]

large_region = ee.Geometry.Polygon(large_extent_coords, proj = 'EPSG:32733', evenOdd = False)

In [7]:
# load Etosha National Park study area
enp = ee.FeatureCollection('WCMC/WDPA/current/polygons') \
        .filter(ee.Filter.eq('ORIG_NAME', 'Etosha'))

# turn ENP study area into a geometry
enp_geom = enp.geometry()

In [8]:
# define sample study area 
# note: sample area is smaller than original sample area because of Landsat data size limit when exporting 
sample_area = ee.Geometry.Polygon(
        [[[15.457215604955591, -18.68807168881119],
          [15.457215604955591, -19.120709491695354],
          [15.939240751439966, -19.120709491695354],
          [15.939240751439966, -18.68807168881119]]], None)

In [28]:
# get output folder path for MODIS images
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
out_dir_m = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/modis/' + operation_m + week)
out_dir_l8 = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/l8/' + operation_l + week)

## Dataset retrieval

In [10]:
## Landsat 8 
# note: ee.List should also normally include 180 (removed it for this sampled study area)
# source: https://developers.google.com/earth-engine/apidocs/ee-date-advance#colab-python
# note: can't filter bounds with large_region until have reprojected image collections (meantime use enp_geom)
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(start_date.advance(l8_buffer.multiply(-1), 'day'), end_date.advance(l8_buffer, 'day')) \
        .filterBounds(enp_geom) 

In [11]:
## MODIS daily 250m
#alternative to filterDate: filter(ee.Filter.inList('system:index', l8_dates)) \
modis_250 = ee.ImageCollection('MODIS/061/MOD09GQ') \
    .filterDate(start_date, end_date) \
    .filterBounds(enp_geom)

# import MODIS 500m dataset (contains information for cloudmasking)
modis_500 = ee.ImageCollection('MODIS/006/MOD09GA') \
        .filterDate(start_date, end_date) \
        .filterBounds(enp_geom)

In [12]:
# combine datasets
# pixel with certain MODIS 500m value can be selected for MODIS 250m 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth
modis_combined = modis_500.combine(modis_250)

# new name of MODIS 250m bands (to avoid having two of similar name)
modisBands = ['sur_refl_b01_1', 'sur_refl_b02_1']
renamedBands = ['red', 'nir']

## Functions

In [13]:
# function to extract and combined relevant QA band bits/flags 
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/
def getQABits(image, start_bit, end_bit, band_name):
    pattern = 0
    
    # for each bit/flag of the QA band --> assign new value to its bits 
    # so each flag will have a different value and the pixel will be the sum of all flags 
    for i in range(start_bit, end_bit):
        pattern += math.pow(2,i)
    # return single band image of extracted QA bits
    # source: https://developers.google.com/earth-engine/apidocs/ee-image-bitwiseand
    return image.select([0], [band_name]) \
                .bitwiseAnd(pattern) \
                .rightShift(start_bit)

In [14]:
# function to mask out cloudy pixels 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/
def maskCloudsMODIS(image):
    # selects the MODIS 500m QA band 
    QA = image.select('state_1km')

    # creates a cloud&shadow flag from specified bits --> in this case: 'Cloud state' and 'Cloud shadow'
    pixelQuality = getQABits(QA, 0, 2, 'cloud_and_shadow_quality_flag')

    # returns mask of cloudy landsat pixels --> cloudy pixels have value = -9999
    # source: https://gis.stackexchange.com/questions/319404/setting-masked-pixel-value-in-google-earth-engine-image-export
    return image.updateMask(pixelQuality.eq(0)).Not().Not().unmask(0)


def maskCloudsLandsat8(image):
    # selects the pixel quality band 
    QA = image.select('QA_PIXEL')

    # creates cloud&shadow flags from specified bits --> in this case: 'Cloud confidence' and 'Cloud shadow confidence'
    cloudConfidence = getQABits(QA, 8, 9, 'cloud_confidence_flag')
    shadowConfidence = getQABits(QA, 11, 11, 'shadow_confidence_flag')

    # returns mask of cloudy landsat pixels --> cloudy pixels have value = -9999
    # source: https://gis.stackexchange.com/questions/319404/setting-masked-pixel-value-in-google-earth-engine-image-export
    return image.updateMask(shadowConfidence.eq(0)).updateMask(cloudConfidence.eq(1)).unmask(-9999)


In [15]:
# define landsat scaling factor
def applyScaleFactors(image): 
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)

In [16]:
# function to reproject (32733 if want Namibia coordinates, but elephant fixes are in 4326)
def reprojectLandsat(image): 
    return image.reproject('EPSG:32733', None, 30)

def reprojectModis(image):
    return image.reproject('EPSG:32733', None, 250)


In [17]:
# function to clip image to study area
def clipToAOI(image): 
    result = image.clip(large_region)
    return result.copyProperties(image, ['system:id'])

In [18]:
# function to calculate NDVI --> NOT NEEDED
def addNDVI(image): 
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return image.addBands(ndvi)

## Cloudmasking MODIS images

In [19]:
# create cloud free MODIS 250m dataset 
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/ 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth

modis_cloudFree = modis_combined.map(maskCloudsMODIS).select(modisBands, renamedBands)

l8_cloudFree = l8.map(maskCloudsLandsat8)

## Rescale Landsat bands and remove band 1

In [22]:
# apply scaling factor 
l8_scaled = l8_cloudFree.map(applyScaleFactors).select(['SR_B.'])

In [23]:
# remove B1 from Landsat 8 image collection 
# source: https://developers.google.com/earth-engine/apidocs/ee-list-remove
l8_scaled = l8_scaled.select(ee.List(l8_scaled.first().bandNames().getInfo()).remove('SR_B1'))

## Reproject and Clip

In [24]:
# apply reprojection 
# cloudmasked
#modis_reproj = modis_cloudFree.map(reprojectModis)
# no cloudmask
#modis_250 = modis_250.select(['sur_refl_b02', 'sur_refl_b01'], ['nir', 'red'])
modis_reproj = modis_cloudFree.map(reprojectModis)
l8_reproj = l8_scaled.map(reprojectLandsat)

In [25]:
# apply clipping function 
modis_clipped = modis_reproj.map(clipToAOI)
l8_clipped = l8_reproj.map(clipToAOI)

In [26]:
# calculate NDVI for MODIS --> CAN REMOVE DONT NEED THIS
#modis_ndvi = modis_clipped.map(addNDVI).select(['NDVI'])

## Visualize

In [18]:
# define visualization parameters
visualization_modis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['nir', 'nir', 'red'],
}

visualization_l8 = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 8000,
  'max': 20000,
}

visualization_l8_scaled = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 0.5,
}

In [22]:
# initialization and set up of the map
Map = geemap.Map()
#Map.add_ee_layer(enp, None, 'Etosha National Park')
#Map.add_layer(sample_area, None, 'Sample AOI')
#Map.add_layer(l8_scaled.first(), visualization_l8_scaled, 'L8 Scaled')
modis = modis_clipped.filterDate('2014-01-19', '2014-01-20')
Map.add_layer(modis_cloudFree.filterDate('2014-01-19', '2014-01-20'), visualization_modis, 'MODIS Cloud Free Jan 19')
Map.add_layer(modis, visualization_modis, 'MODIS final Jan 19')
Map.centerObject(enp_geom)

# Show the map
#Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export data

In [29]:
# export Landsat image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb

#### attempt
#l8_scaledA = l8.map(applyScaleFactors).select(['QA_PIXEL', 'SR_B.'])
#l8_scaledAb = l8_scaledA.select(ee.List(l8_scaled.first().bandNames().getInfo()).remove('SR_B1'))
#l8_reprojA = l8_scaledAb.map(reprojectLandsat)
#l8_clippedA = l8_reprojA.map(clipToAOI)

#print(l8.first().bandNames().getInfo())

# export original image --> check out_dir 
geemap.ee_export_image_collection(l8_clipped, out_dir = out_dir_l8)

# export cloud mask of landsat 8 --> check out_dir
#geemap.ee_export_image_collection(l8_mask_clipped, out_dir = out_dir_l8)

Total number of images: 20

Exporting 1/20: /home/osboxes/Documents/MSc_Thesis/data/l8/cloudmasked/2300/LC08_178073_20140108.tif
Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must not be empty.


Exporting 2/20: /home/osboxes/Documents/MSc_Thesis/data/l8/cloudmasked/2300/LC08_178073_20140124.tif
Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must not be empty.


Exporting 3/20: /home/osboxes/Documents/MSc_Thesis/data/l8/cloudmasked/2300/LC08_178074_20140108.tif
Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must not be empty.


Exporting 4/20: /home/osboxes/Documents/MSc_Thesis/data/l8/cloudmasked/2300/LC08_178074_20140124.tif
Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must not be empty.


Exporting 5/20: /hom

In [32]:
# filter MODIS dates to only take dates of interest, exclude images that were used for gap filling (if gap filled)
#modis = ee.ImageCollection(modis_ndvi).filterDate(start_date, end_date)

# export MODIS image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
geemap.ee_export_image_collection(modis_clipped, out_dir = out_dir_m)

Total number of images: 14

Exporting 1/14: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_23.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_23.tif


Exporting 2/14: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_24.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_24.tif


Exporting 3/14: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_25.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_25.tif


Exporting 4/14: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_26.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2300/2014_01_26.tif


Exporting 5/14: /home/osboxes/Documents/MSc_Thesis/data/modi